<a href="https://colab.research.google.com/github/LyIron23/GitLab/blob/main/challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing the dataset
pokemon = pd.read_csv("/content/drive/MyDrive/Pokemon.csv")

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [ ]:
# your code here

# the sum of 6 fields for each Pokemon
sum_of_abilities = pokemon['HP'] + pokemon['Attack'] + pokemon['Defense'] + pokemon['Sp. Atk'] + pokemon['Sp. Def'] + pokemon['Speed']

# check if the formula works
are_totals_matching = (sum_of_abilities == pokemon['Total']).all()
if are_totals_matching:
    print("Hypothesis: Right ")
else:
    print("Hypothesis: Wrong ")

Hypothesis: Right 


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed.

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [ ]:

#  new columns for each unique value in Type 1 and Type 2
encoded_type1 = pd.get_dummies(pokemon['Type 1'])
encoded_type2 = pd.get_dummies(pokemon['Type 2'])

# concatenate to the original DataFrame
pokemon_encoded = pd.concat([pokemon, encoded_type1, encoded_type2], axis=1)


# drop the original Type 1 and Type 2 columns
pokemon_encoded = pokemon_encoded.drop(['Type 1', 'Type 2'], axis=1)


In [ ]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [ ]:
unique_type1_values = pokemon['Type 1'].unique()
for value in unique_type1_values:
    print(value)

Grass
Fire
Water
Bug
Normal
Poison
Electric
Ground
Fairy
Fighting
Psychic
Rock
Ghost
Ice
Dragon
Dark
Steel
Flying


In [ ]:
correlation_type_fields = {}

for type_field in ['Type_Water', 'Type_Fire', 'Type_Grass', 'Type_Electric', 'Type_Psychic', 'Type_Normal', 'Type_Fighting', 'Type_Ground', 'Type_Rock', 'Type_Ice', 'Type_Flying', 'Type_Poison', 'Type_Bug', 'Type_Ghost', 'Type_Steel', 'Type_Dark', 'Type_Dragon', 'Type_Fairy']:
  correlation = pokemon_encoded['Total'].corr(pokemon_encoded[type_field])
  correlation_type_fields[type_field] = correlation

# Sort the correlations in descending order
sorted_correlations = sorted(correlation_type_fields.items(), key=lambda x: x[1], reverse=True)

# highest correlation will be the first in the sorted list
highest_correlation_type = sorted_correlations[0]

print(f"The Pokemon type with the highest correlation to 'Total' is '{highest_correlation_type[0]}' with a correlation coefficient of {highest_correlation_type[1]}")

KeyError: ignored

In [ ]:
encoded_type_columns = [col for col in pokemon_encoded.columns if col.startswith('Type_')]

# Calculate the correlation between 'Total' and each encoded type field
correlation_type_fields = {}

for type_field in encoded_type_columns:
  correlation = pokemon_encoded['Total'].corr(pokemon_encoded[type_field])
  correlation_type_fields[type_field] = correlation

# Sort the correlations in descending order
sorted_correlations = sorted(correlation_type_fields.items(), key=lambda x: x[1], reverse=True)

# The type with the highest correlation to 'Total' will be the first in the sorted list
highest_correlation_type = sorted_correlations[0]



IndexError: ignored

In [ ]:
encoded_type_columns = [col for col in pokemon_encoded.columns if col.startswith('Type ')]

for type_field in encoded_type_columns:
  correlation = pokemon_encoded['Total'].corr(pokemon_encoded[type_field])
  correlation_type_fields[type_field] = correlation

# Sort the correlations in descending order
sorted_correlations = sorted(correlation_type_fields.items(), key=lambda x: x[1], reverse=True)

highest_correlation_type = sorted_correlations[0]

IndexError: ignored

In [ ]:
encoded_type_columns = [col for col in pokemon_encoded.columns if col.startswith('Type ')]

for type_field in encoded_type_columns:
  correlation = pokemon_encoded['Total'].corr(pokemon_encoded[type_field])
  correlation_type_fields[type_field] = correlation

# Sort the correlations in descending order
sorted_correlations = sorted(correlation_type_fields.items(), key=lambda x: x[1], reverse=True)
if sorted_correlations:
  highest_correlation_type = sorted_correlations[0]
  print(f"The Pokemon type with the highest correlation to 'Total' is '{highest_correlation_type[0]}' with a correlation coefficient of {highest_correlation_type[1]}")
else:
  print("No correlations found.")

No correlations found.


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here